# Reset the Demo
<mark>## TODO - Manual Items</mark>  
Create the following:
* [`audit.SchemaDrift`](/SqlScripts/SchemaDrift/audit.SchemaDrift.sql) - Table to track changes
* [`utfGetTableDefinition`](/SqlScripts/SchemaDrift/utfGetTableDefinition.sql) - User defined function for getting a table's column definitions
* [`uspConformTable`](/SqlScripts/SchemaDrift/uspConformTable.sql) - The stored procedure that does the comparison and optionally changes the schema of the target table
* Assure Data Factory account has rights to execute the procedure, query tables, etc. [Sample](/SqlScripts/SchemaDrift/AssignRights.sql)



Assure the `stage` schema exists  

Drop the `PersonRigid` and `PersonFlexible` target tables and all the staging tables

In [234]:
/* ====================================================================================================================
    [stage] schema
==================================================================================================================== */
if not exists (select * from sys.schemas s where s.name = 'stage')
    execute sp_executesql N'create schema stage';

/* ====================================================================================================================
    [Person...] tables
==================================================================================================================== */
if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'dbo' and t.name = 'PersonRigid')
    drop table dbo.PersonRigid;

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'dbo' and t.name = 'PersonFlexible')
    drop table dbo.PersonFlexible;

/* ====================================================================================================================
    Staging tables
==================================================================================================================== */
if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'stage' and t.name = 'PersonA')
    drop table stage.PersonA

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'stage' and t.name = 'PersonB')
    drop table stage.PersonB

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'stage' and t.name = 'PersonC')
    drop table stage.PersonC

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'stage' and t.name = 'PersonD')
    drop table stage.PersonD

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'stage' and t.name = 'PersonE')
    drop table stage.PersonE


Commands completed successfully.

Total execution time: 00:00:00.061

## Create `PersonRigid` and `PersonFlexible` tables to use as our targets
Start simple
* Id
* First name
* Middle initial
* Last names

In [235]:
/* ====================================================================================================================
    dbo.[Person...] tables
==================================================================================================================== */
create table dbo.PersonRigid
(   Id              int not null identity(1, 1)
,   FirstName       varchar(10)
,   MiddleInitial   char(1)
,   LastName        varchar(15)
)

create table dbo.PersonFlexible
(   Id              int not null identity(1, 1)
,   FirstName       varchar(10)
,   MiddleInitial   char(1)
,   LastName        varchar(15)
)

Commands completed successfully.

Total execution time: 00:00:00.028

## Create Some Stage Tables

### `PersonA`
Same schema as target (except Id) and a few rows that fit nicely into it

In [236]:
/* ====================================================================================================================
    Create table
==================================================================================================================== */
create table stage.PersonA
(   FirstName       varchar(10)
,   MiddleInitial   char(1)
,   LastName        varchar(15)
)

insert into stage.PersonA (FirstName, MiddleInitial, LastName)
select      fn, mi, ln
from    (   values  ('Al',      'B', 'Cook')
            ,       ('Ben',     'C', 'Dunn')
            ,       ('Carl',    'D', 'Ellis')
            ,       ('Dee',     'E', 'Frank')
            ,       ('Elise',   'F', 'Greer')
            ,       ('Fran',    'G', 'Howard')
        )   v(fn, mi, ln)

(6 rows affected)

Total execution time: 00:00:00.031

### `PersonB`

FirstName & LastName wider, but data would still fit old  
_Safe to use with Perform DML = 0_

In [237]:
/* ====================================================================================================================
    Create table
==================================================================================================================== */
create table stage.PersonB
(   FirstName       varchar(20)
,   MiddleInitial   char(1)
,   LastName        varchar(25)
)

insert into stage.PersonB (FirstName, MiddleInitial, LastName)
select      fn, mi, ln
from    (   values  ('Glen',    'H', 'Ibarra')
            ,       ('Hank',    'I', 'James')
            ,       ('Irene',   'J', 'Kent')
            ,       ('Jackie',  'K', 'Lowe')
            ,       ('Kelly',   'L', 'Merchant')
        )   v(fn, mi, ln)

(5 rows affected)

Total execution time: 00:00:00.033

### `PersonC`
Like `PersonB` has wider schema, but this one has longer names  
___NOT___ _Safe to use with Perform DML = 0_

In [238]:
/* ====================================================================================================================
    Create table
==================================================================================================================== */
create table stage.PersonC
(   FirstName       varchar(20)
,   MiddleInitial   char(1)
,   LastName        varchar(25)
)

insert into stage.PersonC (FirstName, MiddleInitial, LastName)
select      fn, mi, ln
from    (   values  ('Mary-Elizabeth',  'N', 'Nelson')
            ,       ('Nathan',          'O', 'Pennyworth-Johnson')
        )   v(fn, mi, ln)

(2 rows affected)

Total execution time: 00:00:00.031

### `PersonD`

A bunch of raw data without a declared schema, this time with MiddleName field

In [239]:
/* ====================================================================================================================
    Create table from a set of values
==================================================================================================================== */
select      FirstName, MiddleName, LastName
into        stage.PersonD
from    (   values  ('Oliver',      'Phillip',        'Quinn')
            ,       ('Penelope',    'Q',              'Robertson')
            ,       ('Quinn',       'Richard',        'Stephenson-MacDonald')
            ,       ('Rebecca',     'Stephanie',      'Thompson-Abercrombie')
            ,       ('Sylvie',      'Theresa',        'Ummmm - I forgot to ask her last name!  What do I do with no last name?')
        )   v(FirstName, MiddleName, LastName);


(5 rows affected)

Total execution time: 00:00:00.031

### `PersonE`

A bunch of raw UNICODE data without a declared schema, this time with MiddleName field

In [240]:
/* ====================================================================================================================
    Create table from a set of values
==================================================================================================================== */
select      FirstName, MiddleName, LastName
into        stage.PersonE
from    (   values  (N'Terry',       N'U',              N'Versacé')
            ,       (N'Umbérto',     N'Vincent',        N'Wilson')
            ,       (N'Viviënne',    N'Wilma',          N'Xávier')
        )   v(FirstName, MiddleName, LastName);


(3 rows affected)

Total execution time: 00:00:00.029

In [241]:
/* ====================================================================================================================
    Add some Hangul Jamo (I don't know any Korean, so this is just some random characters)
==================================================================================================================== */
with num as (
    select      -1 + row_number() over (order by object_id) id
    from        sys.objects
), w as (
    select      top 10 row_number() over (order by num.id) id
            ,   nchar(0x11b0 + num.id) + nchar(0x11c0 + num.id) + nchar(0x11d0 + num.id) + nchar(0x11e0 + num.id) + nchar(0x11f0 + num.id) word
    from        num
)

insert into stage.PersonE(FirstName, MiddleName, LastName)
select      f.word + left(reverse(l.word), 2) FirstName, m.word MiddleName, l.word + substring(m.word, 3, 2) LastName
from        w f
join        w m on m.id = f.id + 3
join        w l on l.id = f.id + 6


(4 rows affected)

Total execution time: 00:00:00.038

## `stage.PersonD` & `stage.PersonE` - schemas generated based on data 

In [242]:
--	==========================================================================
--	Get list of columns in a table (or table family)
--	==========================================================================
select		case when s.name = lag(s.name) over (order by s.name, t.name, c.column_id) then '' else s.name end [schema]
        ,   case when t.name = lag(t.name) over (order by s.name, t.name, c.column_id) then '' else t.name end [table]
        ,   c.column_id, c.name [column], c.is_nullable
        ,	ty.name datatype
        ,   case when ty.name in ('nchar', 'nvarchar', 'ntext') then c.max_length / 2 else c.max_length end max_length
        ,   c.precision, c.scale
from		sys.schemas s
join		sys.tables t on s.schema_id = t.schema_id
join		sys.columns c on t.object_id = c.object_id
join		sys.types ty on ty.system_type_id = c.system_type_id and ty.user_type_id = c.user_type_id
where		t.name like 'Person[D..E]'
    and		c.name like '%'
--	and		s.name like '%'
order by	s.name, t.name, c.column_id

(6 rows affected)

Total execution time: 00:00:00.023

schema,table,column_id,column,is_nullable,datatype,max_length,precision,scale
stage,PersonD,1,FirstName,0,varchar,8,0,0
,,2,MiddleName,0,varchar,9,0,0
,,3,LastName,0,varchar,71,0,0
,PersonE,1,FirstName,0,nvarchar,8,0,0
,,2,MiddleName,0,nvarchar,7,0,0
,,3,LastName,0,nvarchar,7,0,0


Rebuild __`audit.SchemaDrift`__

In [243]:
/* ====================================================================================================================
    Assure needed schemas exist
==================================================================================================================== */
if not exists (select name from sys.schemas where name = 'history')
    execute sp_executesql N'Create schema history';
go

if not exists (select name from sys.schemas where name = 'audit')
    execute sp_executesql N'Create schema audit';
go

/* ====================================================================================================================
    Remove audit.SchemaDrift for rebuild
==================================================================================================================== */
if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'audit' and t.name = 'SchemaDrift' and t.temporal_type_desc = 'SYSTEM_VERSIONED_TEMPORAL_TABLE')
    if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'history' and t.name = 'SchemaDrift' and temporal_type_desc = 'HISTORY_TABLE')
    begin
        alter table audit.SchemaDrift set (system_versioning = off);
        drop table history.SchemaDrift;
        drop table audit.SchemaDrift;
    end

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'history' and t.name = 'SchemaDrift')
    drop table history.SchemaDrift;

if exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'audit' and t.name = 'SchemaDrift')
    drop table audit.SchemaDrift;

/* ====================================================================================================================
    Create the table and its history table
==================================================================================================================== */
if not exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'history' and t.name = 'SchemaDrift')
    create table history.SchemaDrift
        (   SchemaName      sysname
        ,   TableName       sysname
        ,   ColumnName      sysname
        ,   [Definition]    nvarchar(2000)
        ,   PriorDefinition nvarchar(2000)
        ,   ActionReason    varchar(50)
        ,   DdlProposed     nvarchar(2000)
        ,	SysStartTime    datetime2 not null
        ,	SysEndTime      datetime2 not null
        )

if not exists (select s.name sch, t.name tbl from sys.tables t join sys.schemas s on t.schema_id = s.schema_id where s.name = 'audit' and t.name = 'SchemaDrift')
    create table audit.SchemaDrift
        (   SchemaName      sysname
        ,   TableName       sysname
        ,   ColumnName      sysname
        ,   constraint pk_audit_SchemaDrift primary key clustered (SchemaName, TableName, ColumnName)
        ,   [Definition]    nvarchar(2000)
        ,   PriorDefinition nvarchar(2000)
        ,   ActionReason    varchar(50)
        ,   DdlProposed     nvarchar(2000)
        ,	SysStartTime    datetime2 generated always as row start not null
        ,	SysEndTime      datetime2 generated always as row end not null
	    ,	period for SYSTEM_TIME (SysStartTime, SysEndTime)
        )
        with (system_versioning = on (history_table = history.SchemaDrift))

    

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.095